In [288]:
# import modules
import pandas as pd 
import numpy as np 

from sklearn.preprocessing import (PowerTransformer, 
                                   LabelEncoder)
from sklearn.model_selection import train_test_split

In [289]:
# init global variables
train_path = '/kaggle/input/playground-series-s3e26/train.csv'
test_path = '/kaggle/input/playground-series-s3e26/test.csv'

In [290]:
# read datasets
train_df = pd.read_csv(train_path)
train_df.head()

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,999,D-penicillamine,21532,M,N,N,N,N,2.3,316.0,3.35,172.0,1601.0,179.80,63.0,394.0,9.7,3.0,D
1,1,2574,Placebo,19237,F,N,N,N,N,0.9,364.0,3.54,63.0,1440.0,134.85,88.0,361.0,11.0,3.0,C
2,2,3428,Placebo,13727,F,N,Y,Y,Y,3.3,299.0,3.55,131.0,1029.0,119.35,50.0,199.0,11.7,4.0,D
3,3,2576,Placebo,18460,F,N,N,N,N,0.6,256.0,3.50,58.0,1653.0,71.30,96.0,269.0,10.7,3.0,C
4,4,788,Placebo,16658,F,N,Y,N,N,1.1,346.0,3.65,63.0,1181.0,125.55,96.0,298.0,10.6,4.0,C


In [291]:
class GetDummies:
    def __init__(self, name='category', fit=None):
        self.columns = None
        self.fillna_value = None
        self.name = name
        
        if fit is not None: 
            self.fit(fit)
            self.first_fit = True
        else: 
            self.first_fit = False
            
    def fit(self, series):
        self.columns = series.unique()
        self.first_fit = True

    def set_fillna(self, v):
        self.fillna_value = v

    def lst_transform(self, data):
        result = []

        for val in data:
            variants = [0] * len(self.columns)
            not_founded = True

            for i, col in enumerate(self.columns):
                if val == col:
                    variants[i] = 1
                    result.append(variants)
                    not_founded = False
                    break

            if not_founded:
                result.append([self.fillna_value] * len(self.columns))
        return result

    def transform(self, data):
        tr_lst = self.lst_transform(data)

        df_data = {f'{self.name}_{col}': [] for col in self.columns}

        for tr in tr_lst:
            for col, val in zip(self.columns, tr):
                df_data[f'{self.name}_{col}'].append(val)

        return pd.DataFrame(df_data)

    def __call__(self, data):
        if self.first_fit == False: 
            self.fit(data)
            self.first_fit = True
            
        return self.transform(data.to_list())

In [292]:
false_true_cols = ["Sex", "Ascites", "Spiders", "Edema", "Hepatomegaly"]

drug_enc = GetDummies(name = "drug", fit = train_df['Drug'])

status_enc = GetDummies(name = "status", fit = train_df['Status'])

def false_true_cols_(df) -> pd.DataFrame: 
    for i in false_true_cols: 
        if i == 'Sex': 
            df[i] = df[i].apply(lambda l: 1 if l == 'F' else 0)
        else:  
            df[i] = df[i].apply(lambda l: 1 if l == 'N' else 0)
    return df 

def preprocess_y(df): 
    return status_enc(df)

def categorical_cols_(df) -> pd.DataFrame: 
    drug_dummies = drug_enc(df['Drug'])
    df = df.drop('Drug', axis=1)
    df = pd.concat([drug_dummies, df], axis=1)
    
    return df

def transform_numeric_cols_(df) -> pd.DataFrame:
    ...
    return df

def preprocess_x(df) -> pd.DataFrame: 
    df = df.copy()
    
    df = df.drop('id', axis=1)
    df = transform_numeric_cols_(df)
    df = false_true_cols_(df)
    df = categorical_cols_(df)
    
    return df

In [293]:
X = preprocess_x(train_df.drop(['Status'], axis=1))
y = preprocess_y(train_df['Status'])

X_train, X_val, y_train, y_val = train_test_split(X, y, train_size = 0.8)

In [294]:
from tensorflow.data import Dataset
import tensorflow.keras.layers as l
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
import tensorflow as tf

In [295]:
def build_dataset(X, y, batch_size = 16, shuffle = True, prefetch = True): 
    dataset = Dataset.from_tensor_slices((
        X, 
        y['status_C'], 
        y['status_CL'], 
        y['status_D'])
    ).batch(batch_size)
    
    if shuffle: 
        dataset = dataset.shuffle(16)
        
    if prefetch: 
        dataset = dataset.prefetch(tf.data.AUTOTUNE)
        
    return dataset 

In [296]:
def build_model(input_shape): 
    inputs = l.Input(input_shape, name='input')
    
    inputs = l.Dropout(0.3)(inputs)
    x = l.Dense(64, activation = 'linear', name='hidden')(inputs)
    x = l.BatchNormalization()(x)
    outputs = l.Dense(1, activation='sigmoid', name = 'output')(x)
    
    return Model(inputs, outputs)

In [297]:
class MainModel(Model): 
    def __init__(self, input_shape = (19)): 
        super().__init__()
        
        self.cmodel = build_model(input_shape)
        self.clmodel = build_model(input_shape)        
        self.dmodel = build_model(input_shape)        
        
    def compile(self): 
        super().compile()
        
        self.c_optimizer = Adam(lr = 1e-3)
        self.cl_optimizer = Adam(lr= 1e-3)        
        self.d_optimizer = Adam(lr = 1e-3)        
        
        self.loss_fn = BinaryCrossentropy()
        
    def train_step(self, batch_data): 
        x, c, cl, d = batch_data
        
        with tf.GradientTape(persistent = True) as tape: 
            c_ = self.cmodel(x, training = True)
            cl_ = self.clmodel(x, training = True)            
            d_ = self.dmodel(x, training=  True)            
            
            c_loss = self.loss_fn(c, c_)
            cl_loss = self.loss_fn(cl, cl_)            
            d_loss = self.loss_fn(d, d_)   
            
            total = c_loss + cl_loss + d_loss
            
        c_grads = tape.gradient(c_loss, self.cmodel.trainable_variables)
        cl_grads = tape.gradient(cl_loss, self.clmodel.trainable_variables)        
        d_grads = tape.gradient(d_loss, self.dmodel.trainable_variables)        
        
        self.c_optimizer.apply_gradients(zip(c_grads, self.cmodel.trainable_variables))
        self.cl_optimizer.apply_gradients(zip(cl_grads, self.clmodel.trainable_variables))        
        self.d_optimizer.apply_gradients(zip(d_grads, self.dmodel.trainable_variables))     
        
        return {'loss': total}
    
    def test_step(self, batch_data): 
        x, c, cl, d = batch_data
        
        c_ = self.cmodel(x)
        cl_ = self.clmodel(x)            
        d_ = self.dmodel(x)            

        c_loss = self.loss_fn(c, c_)
        cl_loss = self.loss_fn(cl, cl_)            
        d_loss = self.loss_fn(d, d_)   

        total = c_loss + cl_loss + d_loss
        
        return {'loss': total}

In [298]:
train_dataset = build_dataset(X_train, y_train, batch_size = 8)
val_dataset = build_dataset(X_val, y_val, batch_size = 16)

In [299]:
model = MainModel()
model.compile()

es = tf.keras.callbacks.EarlyStopping(patience = 60, min_delta = 1e-5, restore_best_weights = True)

In [300]:
model.fit(train_dataset, epochs = 100, validation_data = val_dataset, callbacks = [es])

Epoch 1/100
791/791 [==============================] - 5s 3ms/step - loss: 1.3330 - val_loss: 1.8257
Epoch 2/100
791/791 [==============================] - 2s 3ms/step - loss: 1.1946 - val_loss: 1.1325
Epoch 3/100
791/791 [==============================] - 2s 3ms/step - loss: 1.1799 - val_loss: 0.8081
Epoch 4/100
791/791 [==============================] - 2s 3ms/step - loss: 1.1726 - val_loss: 0.6942
Epoch 5/100
791/791 [==============================] - 2s 3ms/step - loss: 1.1670 - val_loss: 0.9188
Epoch 6/100
791/791 [==============================] - 2s 3ms/step - loss: 1.1584 - val_loss: 1.3226
Epoch 7/100
791/791 [==============================] - 2s 3ms/step - loss: 1.1477 - val_loss: 1.5972
Epoch 8/100
791/791 [==============================] - 2s 3ms/step - loss: 1.1416 - val_loss: 1.4355
Epoch 9/100
791/791 [==============================] - 2s 3ms/step - loss: 1.1345 - val_loss: 2.1812
Epoch 10/100
791/791 [==============================] - 3s 3ms/step - loss: 1.1240 - val_lo

In [305]:
test_df = pd.read_csv(test_path)
test_X = preprocess_x(test_df)

In [306]:
c_prediction = model.cmodel(test_X.values)
cl_prediction = model.clmodel(test_X.values)
d_prediction = model.dmodel(test_X.values)

df_data = {
          'Status_D': [i[0] for i in d_prediction.numpy()], 
    'Status_C': [i[0] for i in c_prediction.numpy()], 
          'Status_CL': [i[0] for i in cl_prediction.numpy()]} 


In [307]:
submission = pd.DataFrame({'id': test_df['id'], **df_data})

In [308]:
submission.to_csv('submission.csv', index=False)